In [1]:
import pandas as pd
import gensim.models as gsm
import phrase2vec as p2v
from emosent import get_emoji_sentiment_rank

In [2]:
emoji_model = gsm.KeyedVectors.load_word2vec_format('D:\Downloads\DP\emoji2vec300.bin', binary=True)

8476 | INFO | loading projection weights from D:\Downloads\DP\emoji2vec300.bin
8476 | INFO | loaded (1661, 300) matrix from D:\Downloads\DP\emoji2vec300.bin


## Chybova analyza datasetu E2V (DATA-EN-TW)

In [3]:
tweets = pd.read_csv('D:\Downloads\DP\data\e2v_data_emoji.csv')
different = pd.read_csv('D:\Downloads\DP\data\errors_e2v.csv')

In [97]:
different['Label'].value_counts()

Neutral     345
Positive    132
Negative    120
Name: Label, dtype: int64

In [98]:
different['Pred'].value_counts()

Positive    334
Negative    185
Neutral      78
Name: Pred, dtype: int64

### Vyskyt emotikonov 

In [5]:
tweets_neg = tweets[tweets['Label']=='Negative'].reset_index(drop=True)
tweets_neu = tweets[tweets['Label']=='Neutral'].reset_index(drop=True)
tweets_pos = tweets[tweets['Label']=='Positive'].reset_index(drop=True)

different_neg = different[different['Label']=='Negative'].reset_index(drop=True)
different_neu = different[different['Label']=='Neutral'].reset_index(drop=True)
different_pos = different[different['Label']=='Positive'].reset_index(drop=True)

In [7]:
different_neu['Pred'].value_counts()

Positive    238
Negative    107
Name: Pred, dtype: int64

In [31]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

emoji_dic = {}
for i in range (0, len(tweets)):
    tokens = tokenizer.tokenize(tweets['Text'][i])
    for token in tokens:
        if token in emoji_model:
            try:
                emoji_dic[token] += 1
            except:
                emoji_dic[token] = 1
                
diff_dic = {}
for i in range (0, len(different_pos)):
    tokens = tokenizer.tokenize(different_pos['Text'][i])
    for token in tokens:
        if token in emoji_model:
            try:
                diff_dic[token] += 1
            except:
                diff_dic[token] = 1

In [ ]:
# dict(sorted(diff_dic.items(), key=lambda item: item[1], reverse=True))
dict(sorted(emoji_dic.items(), key=lambda item: item[1], reverse=True))

In [32]:
emo_diff = {}
for emo in diff_dic:
    if diff_dic[emo] > 5:
        emo_diff[emo] = diff_dic[emo]/emoji_dic[emo]

dict(sorted(emo_diff.items(), key=lambda item: item[1], reverse=True))

{'💥': 0.11267605633802817,
 '💀': 0.043209876543209874,
 '🙏': 0.040145985401459854,
 '💯': 0.030395136778115502,
 '😏': 0.025735294117647058,
 '😭': 0.02387774594078319,
 '🙌': 0.020930232558139535,
 '😩': 0.01606425702811245,
 '👌': 0.015945330296127564,
 '😂': 0.011603053435114504}

### Pozicie emotikonov

In [25]:
pos = {"beg":0,"mid":0,"end":0}

# for sent in tweets['Text']:
for sent in different['Text']:
    sent_len = len(sent)
    for char in sent[:int(sent_len/3)]:
        if char in emoji_model:
            pos['beg']+=1
            break
    for char in sent[int(sent_len/3):int(2*sent_len/3)]:
        if char in emoji_model:
            pos['mid']+=1
            break
    for char in sent[int(2*sent_len/3):]:
        if char in emoji_model:
            pos['end']+=1
            break
        

In [26]:
pos

{'beg': 1106, 'mid': 2429, 'end': 8802}

### Priemerne hodnoty sentimentu emotikonov

In [6]:
sentiment = 0
count = 0

for sent in tweets_pos['Text']:
    for char in sent:
        if char in emoji_model:
            try:
                sentiment += get_emoji_sentiment_rank(char)["sentiment_score"]
            except:
                pass
            count += 1
            
sentiment /= count
print(f"overall sentiment in dataset: {sentiment}")

overall sentiment in dataset: 0.4601901263105616


In [7]:
tweets

,Id,Label,Text
0,5.120560e+17,Negative,an uh she wanna pillow fight in the middle of ...
1,5.139030e+17,Negative,I miss this girl so much. 😭 http://t.co/XPKWvW...
2,5.149320e+17,Negative,RT @RavenChristiana: RT!!😩“@zahiranichole: So ...
3,5.145400e+17,Negative,I hate seeing parents beat their kids in public 😳
4,5.123840e+17,Positive,💫@fxckinconnor \nLISTENING TO PSYCOBABBLE WHIL...
...,...,...,...
10860,5.145750e+17,Positive,My dream job is to work at FedEx 😂
10861,5.132830e+17,Positive,When @Shauski asks me to take a photo of him w...
10862,5.130820e+17,Negative,@TiaSoSolid @MadiMego 😂😩 you threw that shit up!
10863,5.142620e+17,Negative,The Stupid Things I Do 🙈 I Do It For You 😩😘😍


In [100]:
sentiment = 0
count = 0

for sent in different_neu['Text']:
    for char in sent:
        if char in emoji_model:
            try:
                sentiment += get_emoji_sentiment_rank(char)["sentiment_score"]
            except:
                pass
            count += 1
            
sentiment /= count
print(f"overall sentiment in dataset: {sentiment}")

overall sentiment in dataset: 0.3522329317269079


---

## Chybova analyza datasetu Emocontext (DATA-EN-SE)

In [118]:
tweets = pd.read_csv('D:\Downloads\DP\data\emocontext_emoji.csv')
different = pd.read_csv('D:\Downloads\DP\data\errors_emocontext.csv')

In [34]:
different['Label'].value_counts()

others    114
happy      84
angry      72
sad        51
Name: Label, dtype: int64

### Vyskyt emotikonov 

In [119]:
tweets_sad = tweets[tweets['Label']=='sad'].reset_index(drop=True)
tweets_angry = tweets[tweets['Label']=='angry'].reset_index(drop=True)
tweets_happy = tweets[tweets['Label']=='happy'].reset_index(drop=True)
tweets_others = tweets[tweets['Label']=='others'].reset_index(drop=True)

different_sad = different[different['Label']=='sad'].reset_index(drop=True)
different_angry = different[different['Label']=='angry'].reset_index(drop=True)
different_happy = different[different['Label']=='happy'].reset_index(drop=True)
different_others = different[different['Label']=='others'].reset_index(drop=True)

In [95]:
different_others['Pred'].value_counts()

happy    83
sad      25
angry     6
Name: Pred, dtype: int64

In [120]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

emoji_dic = {}
for i in range (0, len(tweets)):
    tokens = tokenizer.tokenize(tweets['Text'][i])
    for token in tokens:
        if token in emoji_model:
            try:
                emoji_dic[token] += 1
            except:
                emoji_dic[token] = 1
                
diff_dic = {}
for i in range (0, len(different_others)):
    tokens = tokenizer.tokenize(different_others['Text'][i])
    for token in tokens:
        if token in emoji_model:
            try:
                diff_dic[token] += 1
            except:
                diff_dic[token] = 1

In [ ]:
dict(sorted(emoji_dic.items(), key=lambda item: item[1], reverse=True))

In [75]:
emo_diff = {}
for emo in diff_dic:
    if diff_dic[emo] > 5:
        emo_diff[emo] = diff_dic[emo]/emoji_dic[emo]

dict(sorted(emo_diff.items(), key=lambda item: item[1], reverse=True))

{'😍': 0.0634441087613293,
 '😒': 0.055248618784530384,
 '😅': 0.050387596899224806,
 '😘': 0.041666666666666664,
 '😀': 0.03546099290780142,
 '😆': 0.03543307086614173,
 '😂': 0.019834221432800475,
 '😁': 0.015748031496062992,
 '😞': 0.012320328542094456}

### Pozicie emotikonov

In [22]:
model = phrase_model_en.emojiVecModel

In [31]:
pos = {"beg":0,"mid":0,"end":0}

for sent in tweets['Text']:
# for sent in different['Text']:
    sent_len = len(sent)
    for char in sent[:int(sent_len/3)]:
        if char in emoji_model:
            pos['beg']+=1
            break
    for char in sent[int(sent_len/3):int(2*sent_len/3)]:
        if char in emoji_model:
            pos['mid']+=1
            break
    for char in sent[int(2*sent_len/3):]:
        if char in emoji_model:
            pos['end']+=1
            break
        

In [32]:
pos

{'beg': 950, 'mid': 1786, 'end': 4474}

### Priemerne hodnoty sentimentu emotikonov

In [125]:
sentiment = 0
count = 0

for sent in tweets_others['Text']:
    for char in sent:
        if char in emoji_model:
            try:
                sentiment += get_emoji_sentiment_rank(char)["sentiment_score"]
            except:
                pass
            count += 1
            
sentiment /= count
print(f"overall sentiment in dataset: {sentiment}")

overall sentiment in dataset: 0.3077859228971984


In [127]:
sentiment = 0
count = 0

for sent in different_others['Text']:
    for char in sent:
        if char in emoji_model:
            try:
                sentiment += get_emoji_sentiment_rank(char)["sentiment_score"]
            except:
                pass
            count += 1
            
sentiment /= count
print(f"overall sentiment in dataset: {sentiment}")

overall sentiment in dataset: 0.2627545126353791


---

## Chybova analyza datasetu Seesame (DATA-SK-FB)

In [76]:
tweets = pd.read_csv('D:\Downloads\DP\data\seesame_emoji.csv')
different = pd.read_csv('D:\Downloads\DP\data\errors_seesame.csv')

In [77]:
different['Label'].value_counts()

Neutral     2211
Negative     923
Positive     862
Name: Label, dtype: int64

### Vyskyt emotikonov 

In [85]:
tweets_neg = tweets[tweets['Label']=='Negative'].reset_index(drop=True)
tweets_neu = tweets[tweets['Label']=='Neutral'].reset_index(drop=True)
tweets_pos = tweets[tweets['Label']=='Positive'].reset_index(drop=True)

different_neg = different[different['Label']=='Negative'].reset_index(drop=True)
different_neu = different[different['Label']=='Neutral'].reset_index(drop=True)
different_pos = different[different['Label']=='Positive'].reset_index(drop=True)

In [86]:
different_neg['Pred'].value_counts()

Positive    714
Neutral     209
Name: Pred, dtype: int64

In [112]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

emoji_dic = {}
for i in range (0, len(tweets)):
    tokens = tokenizer.tokenize(tweets['Text'][i])
    for token in tokens:
        if token in emoji_model:
            try:
                emoji_dic[token] += 1
            except:
                emoji_dic[token] = 1
                
diff_dic = {}
for i in range (0, len(different_neg)):
    tokens = tokenizer.tokenize(different_neg['Text'][i])
    for token in tokens:
        if token in emoji_model:
            try:
                diff_dic[token] += 1
            except:
                diff_dic[token] = 1

In [ ]:
# dict(sorted(diff_dic.items(), key=lambda item: item[1], reverse=True))
dict(sorted(emoji_dic.items(), key=lambda item: item[1], reverse=True))

In [117]:
emo_diff = {}
for emo in diff_dic:
    if diff_dic[emo] > 50:
        emo_diff[emo] = diff_dic[emo]/emoji_dic[emo]

dict(sorted(emo_diff.items(), key=lambda item: item[1], reverse=True))

{'😄': 0.03812913065582105,
 '😋': 0.03741746148202495,
 '😉': 0.0188957779746088,
 '😃': 0.018885166006701187,
 '🍀': 0.01878952122854562,
 '🙂': 0.017773131207527444,
 '😁': 0.014737991266375546,
 '😀': 0.011796162315193457,
 '👍': 0.007691304914613479,
 '😂': 0.004336754993236254}

### Pozicie emotikonov

In [76]:
pos = {"beg":0,"mid":0,"end":0}

for sent in tweets['Text']:
# for sent in different['Text']:
    sent_len = len(sent)
    for char in sent[:int(sent_len/3)]:
        if char in emoji_model:
            pos['beg']+=1
            break
    for char in sent[int(sent_len/3):int(2*sent_len/3)]:
        if char in emoji_model:
            pos['mid']+=1
            break
    for char in sent[int(2*sent_len/3):]:
        if char in emoji_model:
            pos['end']+=1
            break
        

In [77]:
pos

{'beg': 6225, 'mid': 8447, 'end': 73954}

### Priemerne hodnoty sentimentu emotikonov

In [85]:
sentiment = 0
count = 0

for sent in tweets_pos['Text']:
    for char in sent:
        if char in emoji_model:
            try:
                sentiment += get_emoji_sentiment_rank(char)["sentiment_score"]
            except:
                pass
            count += 1
            
sentiment /= count
print(f"overall sentiment in dataset: {sentiment}")

overall sentiment in dataset: 0.5269538642413389


In [89]:
sentiment = 0
count = 0

for sent in different_pos['Text']:
    for char in sent:
        if char in emoji_model:
            try:
                sentiment += get_emoji_sentiment_rank(char)["sentiment_score"]
            except:
                pass
            count += 1
            
sentiment /= count
print(f"overall sentiment in dataset: {sentiment}")

overall sentiment in dataset: 0.3252052052052057


---

## Testing

In [124]:
# pri EMOCONTEXT su pouzite niektore nove emotikony -> nahradime
emo_mapping = {
    '🙁':'😕',
    '🤣':'😂',
    '🤐':'😬',
    '🙄':'😏',
    '🍾':'🍹',
    '🤗':'☺',
    '🤔':'😏',
    '🤡':'🃏',
    '🤑':'💰',
    '🤥':'😢',
    '🤷':'💁',
    '\u200d':'',
}

for i, j in emo_mapping.items():
    tweets['Text'] = tweets['Text'].str.replace(i, j)
    
for i, j in emo_mapping.items():
    different['Text'] = different['Text'].str.replace(i, j)